In [9]:
#Task 1
'''
Variables
Each step i is represented by a variable which corresponds to a coordinate (x, y) on the grid.

Domains
The domain of each variable is the set of diagonally reachable cells from the previous step, excluding obstacles and visited cells.

Constraints
Diagonal moves only: from (x, y) to (x ± 1, y ± 1)
Within grid bounds: 0 ≤ x < rows, 0 ≤ y < cols
Avoid obstacles
No revisiting the same cell
Start at (1,1), end at (4,4)
Each move cost is √2 (Euclidean distance for diagonal moves)'''

from ortools.sat.python import cp_model
import math

rows, cols = 5, 5
start = (1, 1)
end = (4, 4)
obstacles = set([])

directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]

max_steps = 10

def is_valid(x, y):
    return 0 <= x < rows and 0 <= y < cols and (x, y) not in obstacles

def solve_csp_diagonal_path():
    model = cp_model.CpModel()

    x = [model.NewIntVar(0, rows - 1, f'x_{i}') for i in range(max_steps)]
    y = [model.NewIntVar(0, cols - 1, f'y_{i}') for i in range(max_steps)]

    model.Add(x[0] == start[0])
    model.Add(y[0] == start[1])
    model.Add(x[-1] == end[0])
    model.Add(y[-1] == end[1])

    for i in range(max_steps - 1):
        dx = model.NewIntVar(-1, 1, f'dx_{i}')
        dy = model.NewIntVar(-1, 1, f'dy_{i}')

        model.Add(dx == x[i + 1] - x[i])
        model.Add(dy == y[i + 1] - y[i])

        model.AddAbsEquality(model.NewIntVar(1, 1, ''), dx)
        model.AddAbsEquality(model.NewIntVar(1, 1, ''), dy)

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10.0

    class PathPrinter(cp_model.CpSolverSolutionCallback):
        def __init__(self, x_vars, y_vars):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self.x_vars = x_vars
            self.y_vars = y_vars

        def on_solution_callback(self):
            print("Path:")
            for i in range(max_steps):
                xi = self.Value(self.x_vars[i])
                yi = self.Value(self.y_vars[i])
                print(f"Step {i + 1}: ({xi}, {yi})")
                if (xi, yi) == end:
                    break

    printer = PathPrinter(x, y)
    status = solver.Solve(model, printer)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print(f"Found path with cost: {round(math.sqrt(2) * (max_steps - 1), 2)}")
    else:
        print("No path found.")

solve_csp_diagonal_path()


Path:
Step 1: (1, 1)
Step 2: (0, 0)
Step 3: (1, 1)
Step 4: (0, 0)
Step 5: (1, 1)
Step 6: (0, 0)
Step 7: (1, 1)
Step 8: (2, 2)
Step 9: (3, 3)
Step 10: (4, 4)
Found path with cost: 12.73


In [13]:
#Task 2
from ortools.sat.python import cp_model

def compute_island_perimeter(grid):
    rows, cols = len(grid), len(grid[0])
    model = cp_model.CpModel()

    island = [[model.NewBoolVar(f'island_{r}_{c}') for c in range(cols)] for r in range(rows)]

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 0:
                model.Add(island[r][c] == 0)

    start_found = False
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 1:
                model.Add(island[r][c] == 1)
                start = (r, c)
                start_found = True
                break
        if start_found:
            break

    for r in range(rows):
        for c in range(cols):
            if (r, c) == start or grid[r][c] == 0:
                continue

            neighbors = []
            for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols:
                    neighbors.append(island[nr][nc])
            if neighbors:
                model.AddMaxEquality(island[r][c], neighbors)

    perimeter_terms = []
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 1:
                for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
                    nr, nc = r + dr, c + dc
                    if nr < 0 or nr >= rows or nc < 0 or nc >= cols:
                        perimeter_terms.append(island[r][c])
                    elif grid[nr][nc] == 0:
                        perimeter_terms.append(island[r][c])

    perimeter = model.NewIntVar(0, rows * cols * 4, 'perimeter')
    model.Add(perimeter == sum(perimeter_terms))

    land_area = sum(island[r][c] for r in range(rows) for c in range(cols))
    model.Maximize(land_area)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Max continuous landmass perimeter:", solver.Value(perimeter))
        print("Map of largest landmass:")
        for r in range(rows):
            row_str = ""
            for c in range(cols):
                row_str += str(solver.Value(island[r][c])) + " "
            print(row_str)
    else:
        print("No solution found.")

grid = [
    [0,1,1,0,0],
    [1,1,1,0,0],
    [0,1,0,0,1],
    [0,0,0,1,1],
    [0,0,0,0,0],
]
compute_island_perimeter(grid)

Max continuous landmass perimeter: 20
Map of largest landmass:
0 1 1 0 0 
1 1 1 0 0 
0 1 0 0 1 
0 0 0 1 1 
0 0 0 0 0 


In [17]:
#Task 3
from ortools.sat.python import cp_model
import numpy as np

def solve_tsp_csp(distance_matrix):
    n = len(distance_matrix)
    model = cp_model.CpModel()

    x = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                x[i, j] = model.NewBoolVar(f'x[{i}][{j}]')

    for j in range(n):
        model.AddExactlyOne(x[i, j] for i in range(n) if i != j)

    for i in range(n):
        model.AddExactlyOne(x[i, j] for j in range(n) if i != j)

    u = [model.NewIntVar(0, n - 1, f'u[{i}]') for i in range(n)]
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.Add(u[i] + 1 <= u[j] + (n - 1) * (1 - x[i, j]))

    total_cost = model.NewIntVar(0, 100000, 'total_cost')
    model.Add(total_cost == sum(x[i, j] * distance_matrix[i][j]
                                for i in range(n) for j in range(n) if i != j))
    model.Minimize(total_cost)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        print("Minimum tour cost:", solver.Value(total_cost))
        tour = []
        current_city = 0
        visited = set()
        while len(tour) < n:
            tour.append(current_city)
            visited.add(current_city)
            for j in range(n):
                if current_city != j and solver.Value(x[current_city, j]):
                    current_city = j
                    break
        print("Tour:", tour + [tour[0]])
    else:
        print("No solution found.")

np.random.seed(42)
cities = 10
matrix = np.random.randint(10, 100, size=(cities, cities))
np.fill_diagonal(matrix, 0)
for i in range(cities):
    for j in range(i+1, cities):
        matrix[j][i] = matrix[i][j]

solve_tsp_csp(matrix)


Minimum tour cost: 302
Tour: [0, 4, 7, 6, 8, 1, 2, 5, 9, 3, 0]


In [27]:
#Task 4
from ortools.sat.python import cp_model
import random

GRID_SIZE = 6
NUM_ROBOTS = 5
NUM_PACKAGES = 10
random.seed(42)

robot_capacities = [random.randint(12, 20) for _ in range(NUM_ROBOTS)]
package_weights = [random.randint(2, 8) for _ in range(NUM_PACKAGES)]

package_sources = [(random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)) for _ in range(NUM_PACKAGES)]
package_dests = [(random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)) for _ in range(NUM_PACKAGES)]

model = cp_model.CpModel()

assigned = [model.NewIntVar(0, NUM_ROBOTS - 1, f'assigned_p{p}') for p in range(NUM_PACKAGES)]

is_assigned = {}
for p in range(NUM_PACKAGES):
    for r in range(NUM_ROBOTS):
        is_assigned[p, r] = model.NewBoolVar(f'is_p{p}_r{r}')
        model.Add(assigned[p] == r).OnlyEnforceIf(is_assigned[p, r])
        model.Add(assigned[p] != r).OnlyEnforceIf(is_assigned[p, r].Not())

for r in range(NUM_ROBOTS):
    total_weight = sum(package_weights[p] * is_assigned[p, r] for p in range(NUM_PACKAGES))
    model.Add(total_weight <= robot_capacities[r])

total_cost = model.NewIntVar(0, 1000, "total_cost")
distances = []
for p in range(NUM_PACKAGES):
    sx, sy = package_sources[p]
    dx, dy = package_dests[p]
    dist = abs(sx - dx) + abs(sy - dy)
    distances.append(sum(is_assigned[p, r] * dist for r in range(NUM_ROBOTS)))

model.Add(total_cost == sum(distances))
model.Minimize(total_cost)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Delivery Plan:\n")
    for p in range(NUM_PACKAGES):
        assigned_robot = solver.Value(assigned[p])
        print(f"Package {p} (weight {package_weights[p]})")
        print(f"Source: {package_sources[p]}, Dest: {package_dests[p]}")
        print(f"Assigned to Robot {assigned_robot}")
    print("\nRobot Capacities:")
    for r in range(NUM_ROBOTS):
        cap = robot_capacities[r]
        carried = sum(package_weights[p] for p in range(NUM_PACKAGES) if solver.Value(assigned[p]) == r)
        print(f"Robot {r}: Capacity {cap}, Load {carried}")
    print(f"\nTotal Delivery Cost (sum of distances): {solver.Value(total_cost)}")
else:
    print("No feasible solution found.")


Delivery Plan:

Package 0 (weight 3)
Source: (0, 0), Dest: (5, 3)
Assigned to Robot 0
Package 1 (weight 7)
Source: (1, 1), Dest: (2, 2)
Assigned to Robot 4
Package 2 (weight 2)
Source: (4, 4), Dest: (1, 1)
Assigned to Robot 2
Package 3 (weight 7)
Source: (0, 4), Dest: (2, 0)
Assigned to Robot 3
Package 4 (weight 7)
Source: (1, 5), Dest: (0, 3)
Assigned to Robot 3
Package 5 (weight 6)
Source: (5, 5), Dest: (0, 2)
Assigned to Robot 4
Package 6 (weight 2)
Source: (4, 3), Dest: (2, 4)
Assigned to Robot 4
Package 7 (weight 6)
Source: (1, 3), Dest: (2, 0)
Assigned to Robot 2
Package 8 (weight 5)
Source: (4, 2), Dest: (5, 3)
Assigned to Robot 1
Package 9 (weight 2)
Source: (0, 1), Dest: (4, 0)
Assigned to Robot 2

Robot Capacities:
Robot 0: Capacity 13, Load 3
Robot 1: Capacity 12, Load 5
Robot 2: Capacity 16, Load 10
Robot 3: Capacity 15, Load 14
Robot 4: Capacity 15, Load 15

Total Delivery Cost (sum of distances): 47


In [47]:

from ortools.sat.python import cp_model

def solve_sudoku_with_constraints(grid):
    model = cp_model.CpModel()
    cell = [[model.NewIntVar(1, 9, f'cell_{i}_{j}') for j in range(9)] for i in range(9)]


    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                model.Add(cell[i][j] == grid[i][j])

    for i in range(9):
        model.AddAllDifferent(cell[i])
        model.AddAllDifferent([cell[r][i] for r in range(9)])  # Columns


    for bi in range(3):
        for bj in range(3):
            subgrid = []
            for i in range(3):
                for j in range(3):
                    subgrid.append(cell[bi*3 + i][bj*3 + j])
            model.AddAllDifferent(subgrid)


    main_diag_sum = sum([cell[i][i] for i in range(9)])
    anti_diag_sum = sum([cell[i][8 - i] for i in range(9)])
    main_diag_mod = model.NewIntVar(0, 2, "main_diag_mod")
    anti_diag_mod = model.NewIntVar(0, 2, "anti_diag_mod")
    model.AddModuloEquality(main_diag_mod, main_diag_sum, 3)
    model.Add(main_diag_mod == 0)
    model.AddModuloEquality(anti_diag_mod, anti_diag_sum, 3)
    model.Add(anti_diag_mod == 0)


    primes = [2, 3, 5, 7]
    for i in range(9):
        for j in range(9):
            curr = cell[i][j]
            for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                ni, nj = i + dx, j + dy
                if 0 <= ni < 9 and 0 <= nj < 9:
                    neighbor = cell[ni][nj]
                    for p1 in primes:
                        for p2 in primes:
                            b1 = model.NewBoolVar(f'is_{i}_{j}_{p1}')
                            b2 = model.NewBoolVar(f'is_{ni}_{nj}_{p2}')
                            model.Add(curr == p1).OnlyEnforceIf(b1)
                            model.Add(curr != p1).OnlyEnforceIf(b1.Not())
                            model.Add(neighbor == p2).OnlyEnforceIf(b2)
                            model.Add(neighbor != p2).OnlyEnforceIf(b2.Not())
                            model.AddImplication(b1, b2.Not())

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 30.0
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("\n✅ Sudoku solution with custom constraints:\n")
        for i in range(9):
            row = [solver.Value(cell[i][j]) for j in range(9)]
            print(" ".join(str(num) for num in row))
    else:
        print("❌ No solution found.")



grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

solve_sudoku_with_constraints(grid)


❌ No solution found.
